In [ ]:
import random
from collections import defaultdict

class LockCoverSolver:
    def __init__(self):
        self.universe = set(range(1000))
        # 预计算覆盖表：key是尝试的密码，value是它能覆盖的所有密码集合
        # 这样可以将复杂度从 O(N^3) 降低，加速计算
        self.coverage_map = self._precompute_coverage()
        
    def _is_covered(self, attempt_int, secret_int):
        """
        判断 attempt 是否能打开 secret
        条件：至少有两位数字相同
        """
        s_a = f"{attempt_int:03d}"
        s_s = f"{secret_int:03d}"
        matches = sum(1 for i in range(3) if s_a[i] == s_s[i])
        return matches >= 2

    def _precompute_coverage(self):
        print("正在预计算覆盖关系矩阵 (1000x1000)...")
        mapping = defaultdict(set)
        for attempt in range(1000):
            for secret in range(1000):
                if self._is_covered(attempt, secret):
                    mapping[attempt].add(secret)
        print("预计算完成。")
        return mapping

    def verify_naive_solution(self):
        """
        验证工程解法：固定最后一位为0，遍历前两位 (000, 010, ... 990)
        """
        print("\n--- 1. 验证 '工程派' 100次解法 ---")
        naive_attempts = [i * 10 for i in range(100)] # [000, 010, 020, ..., 990]
        
        covered_secrets = set()
        for attempt in naive_attempts:
            covered_secrets.update(self.coverage_map[attempt])
        
        if len(covered_secrets) == 1000:
            print(f"✅ 验证成功：100次尝试完全覆盖了所有情况。")
        else:
            print(f"❌ 验证失败：仅覆盖了 {len(covered_secrets)} 种情况。")

    def solve_greedy(self, iterations=5):
        """
        使用贪心算法寻找近似最优解（接近下界）。
        由于贪心算法是局部最优，我们运行多次取最好结果。
        """
        print("\n--- 2. 使用贪心算法寻找更优构造 (近似下界) ---")
        best_solution = list(range(1000)) # 初始设为全集
        
        for i in range(iterations):
            current_solution = []
            remaining_universe = self.universe.copy()
            
            # 当还有未被覆盖的密码时
            while remaining_universe:
                best_candidate = -1
                best_cover_count = -1
                best_covered_set = set()
                
                # 策略：在所有可能的候选者中，找到一个能覆盖“当前剩余未覆盖集合”中元素最多的那个
                # 随机打乱候选顺序以避免陷入同一局部最优
                candidates = list(range(1000))
                random.shuffle(candidates)
                
                for candidate in candidates:
                    # 计算该候选者能覆盖多少个“剩余”的密码
                    # 优化：利用集合交集
                    effectively_covered = self.coverage_map[candidate] & remaining_universe
                    count = len(effectively_covered)
                    
                    if count > best_cover_count:
                        best_cover_count = count
                        best_candidate = candidate
                        best_covered_set = effectively_covered
                
                # 采纳这个候选者
                current_solution.append(best_candidate)
                remaining_universe -= best_covered_set
            
            print(f"第 {i+1} 次迭代找到的解长度: {len(current_solution)}")
            if len(current_solution) < len(best_solution):
                best_solution = current_solution

        print(f"\n🏆 找到的最佳构造次数: {len(best_solution)}")
        print(f"理论下界参考: ~36")
        print(f"最佳尝试序列示例 (前10个): {[f'{x:03d}' for x in best_solution[:10]]} ...")
        return best_solution

# 运行程序
if __name__ == "__main__":
    solver = LockCoverSolver()
    
    # 1. 验证 100 次解法
    solver.verify_naive_solution()
    
    # 2. 测算构造更优解
    best_set = solver.solve_greedy(iterations=1000)

正在预计算覆盖矩阵 (Packed uint64)...
预计算完成。

--- Numba 加速模式 (Iterations: 500) ---


In [1]:
import random
import math
import time

class LockCoverAnnealing:
    def __init__(self):
        self.num_points = 1000
        # 预计算覆盖掩码 (同之前的优化)
        self.coverage_masks = self._precompute_coverage_masks()
        self.full_mask = (1 << self.num_points) - 1

    def _is_covered(self, attempt_int, secret_int):
        a1, r = divmod(attempt_int, 100); a2, a3 = divmod(r, 10)
        s1, r = divmod(secret_int, 100); s2, s3 = divmod(r, 10)
        matches = (a1==s1) + (a2==s2) + (a3==s3)
        return matches >= 2

    def _precompute_coverage_masks(self):
        print("正在预计算覆盖位掩码...")
        masks = [0] * self.num_points
        for attempt in range(self.num_points):
            mask = 0
            for secret in range(self.num_points):
                if self._is_covered(attempt, secret):
                    mask |= (1 << secret)
            masks[attempt] = mask
        return masks

    def calculate_uncovered_count(self, solution_indices):
        """计算当前解法有多少个点没被覆盖"""
        # 将所有选中的点的掩码进行 OR 运算
        current_mask = 0
        for idx in solution_indices:
            current_mask |= self.coverage_masks[idx]
        
        # 取反并与全集做与运算，得到未覆盖部分的掩码
        uncovered_mask = (~current_mask) & self.full_mask
        return uncovered_mask.bit_count() # Python 3.10+

    def solve_annealing(self, start_size=58, target_size=40, max_steps_per_level=50000):
        """
        从 start_size 开始，尝试不断删减点数，直到 target_size
        """
        print(f"\n--- 启动模拟退火 (Target: {target_size}) ---")
        
        # 初始化：先随机拿 start_size 个点
        current_solution = random.sample(range(1000), start_size)
        best_overall_solution = list(current_solution)
        
        current_size = start_size
        
        while current_size >= target_size:
            print(f"\n>>> 正在尝试寻找 {current_size} 个点的解...")
            
            # 初始状态
            cost = self.calculate_uncovered_count(current_solution)
            
            # 温度参数
            T = 1.0
            T_min = 0.001
            alpha = 0.9995 # 降温系数
            
            step = 0
            found = False
            
            start_time = time.time()
            
            while step < max_steps_per_level:
                # 1. 产生新解：随机交换一个点
                # 从当前解中随机移除一个
                remove_idx = random.randrange(current_size)
                old_point = current_solution[remove_idx]
                
                # 从剩余池中随机选一个
                new_point = random.randint(0, 999)
                while new_point in current_solution:
                    new_point = random.randint(0, 999)
                
                # 临时替换
                current_solution[remove_idx] = new_point
                
                # 2. 计算新成本 (增量计算会更快，这里为了代码简洁直接重算)
                new_cost = self.calculate_uncovered_count(current_solution)
                
                # 3. 接受准则
                delta = new_cost - cost
                
                if new_cost == 0:
                    # 找到了！完全覆盖！
                    found = True
                    best_overall_solution = list(current_solution)
                    print(f"✅ 成功找到 {current_size} 的解！(耗时 {time.time()-start_time:.2f}s)")
                    break
                
                if delta < 0:
                    # 变好了，接受
                    cost = new_cost
                else:
                    # 变差了，以概率接受 (Metropolis准则)
                    p = math.exp(-delta / T)
                    if random.random() < p:
                        cost = new_cost
                    else:
                        # 拒绝，还原
                        current_solution[remove_idx] = old_point
                
                T = max(T_min, T * alpha)
                step += 1
                
                if step % 5000 == 0:
                    print(f"   Step {step}, Cost(未覆盖数): {cost}, Temp: {T:.4f}")

            if found:
                # 成功找到当前 size 的解，准备挑战更小的 size
                # 策略：直接删掉列表里的最后一个，作为下一轮的初值
                current_solution.pop() 
                current_size -= 1
            else:
                print(f"❌ 在 {max_steps_per_level} 步内未能找到 {current_size} 的解。")
                print("   尝试重新随机初始化本层...")
                # 如果卡住了，可以重置，或者就此停止
                # 这里简单策略：重新随机生成一组 current_size 的初值再试一次
                current_solution = random.sample(range(1000), current_size)
                # 或者直接退出
                # break 
        
        print(f"\n🏆 最终找到的最佳构造大小: {len(best_overall_solution)}")
        return best_overall_solution

if __name__ == "__main__":
    solver = LockCoverAnnealing()
    # 如果想挑战极限 40，可能需要把 max_steps_per_level 调大到 200000+
    best_set = solver.solve_annealing(start_size=60, target_size=40, max_steps_per_level=1000000)
    
    print("样例解:", best_set[:10])

正在预计算覆盖位掩码...

--- 启动模拟退火 (Target: 40) ---

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 33, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 23, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 31, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 1, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 33, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 30, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 39, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 29, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 40, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 32, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数): 39, Temp: 0.0820
   Step 10000, Cost(未覆盖数): 34, Temp: 0.0067
❌ 在 10000 步内未能找到 60 的解。
   尝试重新随机初始化本层...

>>> 正在尝试寻找 60 个点的解...
   Step 5000, Cost(未覆盖数

KeyboardInterrupt: 